In [1]:
import os,shutil
from glob import glob

In [10]:
region="off"
filt="f160w"

# autoset catalog path based on user
if os.environ['USER'] =='remy':
    workdir="/Users/remy/cv/magellanic/n159/"
    dolbin="/Users/remy/cv/magellanic/dolphot/dolphot2.0/bin/"
    
os.chdir(workdir+region+"/"+filt+"/")

if filt=="f125w" or filt=="f160w":
    kind="flt"  
    camera="wfc3"
else:
    # this is of course particular to this project that we did UVO with ACS and IR with WFC3
    kind="flc"
    camera="acs"
    

In [11]:
rawfiles=glob("*"+kind+".fits")
print(rawfiles)

# save a copy of the output from align_drizzle, because wfc3mask is going to overwrite the flc files
for f in rawfiles:
    f0=f[:-5]+".gaiareg.fits"
    if not os.path.exists(f0):
        shutil.copy(f,f0)
        print(camera+"mask "+f)
        os.system(dolbin+camera+"mask "+f)
    else:
        print(f0+" exists")
        
    chipims=glob(f[:-5]+".chip?.fits")
    if len(chipims)<1:
        print("splitgroups "+f)
        os.system(dolbin+"splitgroups "+f)
        chipims=glob(f[:-5]+".chip?.fits")
   
    for fc in chipims:
        fsky=fc[:-5]+".sky.fits"
        if os.path.exists(fsky):
            print(fsky+" exists")
        else:
            print("calcsky "+fc[:-5])
            if camera=="wfc3":
                os.system(dolbin+"calcsky "+fc[:-5]+" 10 25 -64 2.25 2")  # wfc3IR recommended settings
            else:
                os.system(dolbin+"calcsky "+fc[:-5]+" 15 35 -128 2.25 2")  # 
    

# I use a single-HDU drz file - we can use "splitgroups" to remove the bintable
    

['icbna6jqq_flt.fits', 'icbna6jaq_flt.fits', 'icbna6jjq_flt.fits', 'icbna6jgq_flt.fits']
wfc3mask icbna6jqq_flt.fits
splitgroups icbna6jqq_flt.fits
calcsky icbna6jqq_flt.chip1
wfc3mask icbna6jaq_flt.fits
splitgroups icbna6jaq_flt.fits
calcsky icbna6jaq_flt.chip1
wfc3mask icbna6jjq_flt.fits
splitgroups icbna6jjq_flt.fits
calcsky icbna6jjq_flt.chip1
wfc3mask icbna6jgq_flt.fits
splitgroups icbna6jgq_flt.fits
calcsky icbna6jgq_flt.chip1


In [12]:
dpfile="dolparms_"+filt+".txt"
if os.path.exists(dpfile):
    print(dpfile+" exists")
else:
    f=open(dpfile,'w')
    pp=[
        "PSFPhot = 1",
        "PSFPhotIt = 2",
        "Force1 = 0",
        "FitSky = 2",
        "img_RAper = 2",
        "img_RChi = 1.5",
        "img_RPSF = 15",
        "SkySig = 2.25",
        "SecondPass = 5",
        "SigFindMult = 0.85",
        "MaxIT = 25",
        "NoiseMult = 0.10",
        "FSat = 0.999",
        "ApCor = 1",
        "RCentroid = 2",
        "PosStep = 0.25",
        "RCombine = 1.5",
        "SigPSF = 5.0",
        "PSFres = 1",
        "img_shift = 0 0",
        "img_xform = 1 0 0",
        "UseWCS = 1",
    ]
    for p in pp:
        f.write(p+"\n")
    if camera=="acs":
        pp=[
            "img_RSky2 = 4 10",
            "img_RSky = 15 35",
            "img_apsky = 15 25",
            "SkipSky = 2",
            "ACSpsfType = 1",
            "ACSuseCTE = 0",
            "Align = 2"
        ]
    else: # WFCIR
        pp=[    
            "img_RSky2 = 3 10",
            "img_RSky = 8 20",
            "img_apsky = 8 20",
            "SkipSky = 1",
            "WFC3IRpsfType = 1",
            "Align = 1"
        ]
    for p in pp:
        f.write(p+"\n")
    f.close()


dpfile="dolparms_"+filt+".multi.txt"
if os.path.exists(dpfile):
    print(dpfile+" exists")
else:
    f=open(dpfile,'w')
    chipims=sorted(glob("*"+kind+".chip?.fits"))
    f.write("Nimg = %i\n"%len(chipims))
    if camera=="acs":
        # this would use its own drz image as reference:
        # f.write("img_file = "+filt+"_drc_sci\n") 
        # this uses 814 for UVIS - it needs to have just the image split off as "_hdu"
        f.write("img_file = ../f814w/f814w_drc_sci_hdu\n")
    else:
        # this would use its own drz image as reference:
        # f.write("img_file = "+filt+"_drz_sci\n") 
        # this uses 160 for IR - it needs to have just the image split off as "_hdu"
        f.write("img_file = ../f160w/f160w_drz_sci_hdu\n")
    for i,fc in enumerate(chipims):
        f.write(("img%i_file = "%(i+1))+fc[:-5]+"\n")
    f.close()
    

In [13]:
if camera=="acs":
    outroot = region+"_"+filt+"_f814ref"
else:
    outroot = region+"_"+filt+"_f160ref"
if not os.path.exists(outroot):
    cmd=dolbin+"dolphot "+outroot+" -pdolparms_"+filt+".txt "+"-pdolparms_"+filt+".multi.txt > "+outroot+".log"
    print(cmd)
    os.system(cmd)

/Users/remy/cv/magellanic/dolphot/dolphot2.0/bin/dolphot off_f160w_f160ref -pdolparms_f160w.txt -pdolparms_f160w.multi.txt > off_f160w_f160ref.log
